In [1]:
import os
import sys
import pandas as pd
import cv2 as cv
import matplotlib.pyplot as plt
import numpy as np

In [2]:
unpickled_df = pd.read_pickle("synced_full_data.pkl")  
unpickled_df  

,timestamps,detections,img_msgs,vox_msgs
0,1.616680e+09,"[[0, [-0.38936211035001733, -0.027513600453769...","[[[4, 6, 24], [4, 6, 24], [5, 4, 28], [4, 3, 2...","[[5, 20], [15, 33], [12, 29], [11, 27], [19, 3..."
1,1.616680e+09,"[[1, [2.434882329030057, 0.008162512861874015,...","[[[4, 3, 27], [3, 2, 26], [3, 7, 21], [4, 8, 2...","[[3, 78], [24, 38], [19, 53], [1, 78], [16, 32..."
2,1.616680e+09,"[[0, [-0.40092064309254183, -0.021737212341631...","[[[7, 6, 26], [8, 7, 27], [7, 6, 26], [6, 4, 2...","[[6, 77], [16, 60], [46, 33], [2, 82], [26, 32..."
3,1.616680e+09,"[[1, [2.383170036323165, 0.008162512861874015,...","[[[5, 4, 28], [4, 3, 27], [7, 8, 19], [6, 6, 1...","[[25, 79], [24, 58], [23, 58], [21, 32], [23, ..."
4,1.616680e+09,"[[0, [-0.3997900988003534, -0.0190707714418641...","[[[6, 11, 19], [5, 10, 18], [6, 9, 20], [6, 9,...","[[25, 60], [53, 56], [24, 88], [27, 55], [25, ..."
...,...,...,...,...
1041,1.616680e+09,"[[279, [-3.2723291423496175, -0.21280036753926...","[[[124, 114, 112], [125, 115, 114], [129, 114,...","[[37, 70], [34, 56], [35, 61], [35, 37], [37, ..."
1042,1.616680e+09,"[[283, [-2.485738105996217, -0.100405655615125...","[[[221, 215, 192], [219, 212, 190], [210, 193,...","[[37, 70], [34, 56], [35, 61], [35, 37], [37, ..."
1043,1.616680e+09,"[[277, [3.7260399079066175, -0.623008068033155...","[[[253, 255, 253], [252, 255, 252], [213, 204,...","[[37, 70], [34, 56], [35, 61], [35, 37], [37, ..."
1044,1.616680e+09,"[[0.9985901713371277, [3.078814208471769, -0.6...","[[[255, 255, 255], [255, 255, 255], [255, 255,...","[[37, 70], [34, 56], [35, 61], [35, 37], [37, ..."


In [4]:
def draw_boxes(frame, detections):
    # Draw bounding boxes for tracked objects
    for object in detections:
        if type(object[0])== int:
            x, y, w, h = object[2]
            x, y, w, h = int(x), int(y), int(w), int(h)
            object_id = object[0]
            confidence = object[-1]
            frame = cv.rectangle(frame, (x, y), (w, h), (0, 255, 0), 2)
            frame = cv.putText(frame, f"{str(object_id)}: {str(round(confidence, 2))}", (x, y - 10), cv.FONT_HERSHEY_PLAIN, 2, (0, 255, 0), 2)
    return frame

In [5]:
# filter trajectories with enough detected keypoints and length of sequence
def filter_detections_id_pose(index, row):
    old_detections = row["detections"]
    new_detections = []
    for object in old_detections:
        if type(object[0])== int:
            object_id = object[0]
            confidence = object[-1]
            pose_kps = object[3]
            pose_confs = np.asarray(pose_kps)[:,-1]
            if len(np.argwhere(pose_confs > 0.5)) > 8 and not type(confidence) == list:
                if confidence > 0.7:
                    new_detections.append([object_id, object[1], object[2], pose_kps, object[4]])
    unpickled_df.update(pd.DataFrame({'detections': [new_detections]}, index=[index]))


In [6]:
for index, row in unpickled_df.iterrows():
    filter_detections_id_pose(index, row)
    img = row["img_msgs"].astype(np.uint8)
    img = draw_boxes(img, row["detections"])
    cv.imshow("frame", img)
    cv.waitKey(50)

cv.destroyAllWindows()

In [7]:
def filter_seq_len():
    ids = []
    detections = unpickled_df.iloc[:]["detections"]
    all_ids = [x[0] for y in detections for x in y]
    occurs_id, occurs_counts = np.unique(np.asarray(all_ids), return_counts=True)
    occurs_idx = np.argwhere(occurs_counts>10) # TODO: parameter, gucekn wie viel übrig bleibt
    ids = occurs_id[occurs_idx]
    return np.squeeze(ids)
keep_ids = filter_seq_len()
print(keep_ids)

[  0   1   5  25  44  46  59  65  67  72  75  78  81  88  92 105 117 130
 134 144 149 150 151 159 209 227 234 236 238 239 243 249 254 255 259 260
 262]


In [8]:
def filter_ids(ids):
    old_detections = row["detections"]
    new_detections = []
    for object in old_detections:
        if np.isin(object[0], ids):
            object_id = object[0]
            confidence = object[-1]
            pose_kps = object[3]
            pose_confs = np.asarray(pose_kps)[:,-1]
            if len(np.argwhere(pose_confs > 0.5)) > 8:
                if confidence > 0.7:
                    new_detections.append([object_id, object[1], object[2], pose_kps, object[4]])
    unpickled_df.update(pd.DataFrame({'detections': [new_detections]}, index=[index]))

In [9]:
for index, row in unpickled_df.iterrows():
    filter_ids(keep_ids)

In [10]:
for index, row in unpickled_df.iterrows():
    img = row["img_msgs"].astype(np.uint8)
    img = draw_boxes(img, row["detections"])
    cv.imshow("frame", img)
    cv.waitKey(50)

cv.destroyAllWindows()

# Full process of filtering

In [20]:
def draw_boxes(frame, detections):
    # Draw bounding boxes for tracked objects
    for object in detections:
        if type(object[0])== int:
            x, y, w, h = object[2]
            x, y, w, h = int(x), int(y), int(w), int(h)
            object_id = object[0]
            confidence = object[-1]
            frame = cv.rectangle(frame, (x, y), (w, h), (0, 255, 0), 2)
            frame = cv.putText(frame, f"{str(object_id)}: {str(round(confidence, 2))}", (x, y - 10), cv.FONT_HERSHEY_PLAIN, 2, (0, 255, 0), 2)
    return frame

# filter trajectories with enough detected keypoints and length of sequence
def filter_detections_id_pose(index, row):
    old_detections = row["detections"]
    new_detections = []
    for object in old_detections:
        if type(object[0])== int:
            object_id = object[0]
            confidence = object[-1]
            pose_kps = object[3]
            pose_confs = np.asarray(pose_kps)[:,-1]
            if len(np.argwhere(pose_confs > 0.5)) > 8 and not type(confidence) == list:
                if confidence > 0.7:
                    new_detections.append([object_id, object[1], object[2], pose_kps, object[4]])
    unpickled_df.update(pd.DataFrame({'detections': [new_detections]}, index=[index]))

def filter_seq_len():
    ids = []
    detections = unpickled_df.iloc[:]["detections"]
    all_ids = [x[0] for y in detections for x in y]
    occurs_id, occurs_counts = np.unique(np.asarray(all_ids), return_counts=True)
    occurs_idx = np.argwhere(occurs_counts>10) # TODO: parameter, gucekn wie viel übrig bleibt
    ids = occurs_id[occurs_idx]
    return np.squeeze(ids)

def filter_ids(ids):
    old_detections = row["detections"]
    new_detections = []
    for object in old_detections:
        if np.isin(object[0], ids):
            object_id = object[0]
            confidence = object[-1]
            pose_kps = object[3]
            pose_confs = np.asarray(pose_kps)[:,-1]
            new_detections.append([object_id, object[1], object[2], pose_kps, object[4]])
    unpickled_df.update(pd.DataFrame({'detections': [new_detections]}, index=[index]))

In [16]:
# load data
unpickled_df = pd.read_pickle("synced_full_data.pkl")  


for index, row in unpickled_df.iterrows():
    filter_detections_id_pose(index, row)

keep_ids = filter_seq_len()

for index, row in unpickled_df.iterrows():
    filter_ids(keep_ids)

for index, row in unpickled_df.iterrows():
    img = row["img_msgs"].astype(np.uint8)
    img = draw_boxes(img, row["detections"])
    cv.imshow("frame", img)
    cv.waitKey(50)

cv.destroyAllWindows()

In [21]:
### version 2: keep sequences in full length

unpickled_df = pd.read_pickle("synced_full_data.pkl")  



for index, row in unpickled_df.copy().iterrows():
    filter_detections_id_pose(index, row)

keep_ids = filter_seq_len()

for index, row in unpickled_df.iterrows():
    filter_ids(keep_ids)

for index, row in unpickled_df.iterrows():
    img = row["img_msgs"].astype(np.uint8)
    img = draw_boxes(img, row["detections"])
    cv.imshow("frame", img)
    cv.waitKey(50)

cv.destroyAllWindows()

In [22]:
unpickled_df

,timestamps,detections,img_msgs,vox_msgs
0,1.616680e+09,"[[0, [-0.38936211035001733, -0.027513600453769...","[[[4, 6, 24], [4, 6, 24], [5, 4, 28], [4, 3, 2...","[[5, 20], [15, 33], [12, 29], [11, 27], [19, 3..."
1,1.616680e+09,"[[1, [2.434882329030057, 0.008162512861874015,...","[[[4, 3, 27], [3, 2, 26], [3, 7, 21], [4, 8, 2...","[[3, 78], [24, 38], [19, 53], [1, 78], [16, 32..."
2,1.616680e+09,"[[0, [-0.40092064309254183, -0.021737212341631...","[[[7, 6, 26], [8, 7, 27], [7, 6, 26], [6, 4, 2...","[[6, 77], [16, 60], [46, 33], [2, 82], [26, 32..."
3,1.616680e+09,"[[1, [2.383170036323165, 0.008162512861874015,...","[[[5, 4, 28], [4, 3, 27], [7, 8, 19], [6, 6, 1...","[[25, 79], [24, 58], [23, 58], [21, 32], [23, ..."
4,1.616680e+09,"[[0, [-0.3997900988003534, -0.0190707714418641...","[[[6, 11, 19], [5, 10, 18], [6, 9, 20], [6, 9,...","[[25, 60], [53, 56], [24, 88], [27, 55], [25, ..."
...,...,...,...,...
1041,1.616680e+09,[],"[[[124, 114, 112], [125, 115, 114], [129, 114,...","[[37, 70], [34, 56], [35, 61], [35, 37], [37, ..."
1042,1.616680e+09,[],"[[[221, 215, 192], [219, 212, 190], [210, 193,...","[[37, 70], [34, 56], [35, 61], [35, 37], [37, ..."
1043,1.616680e+09,[],"[[[253, 255, 253], [252, 255, 252], [213, 204,...","[[37, 70], [34, 56], [35, 61], [35, 37], [37, ..."
1044,1.616680e+09,[],"[[[255, 255, 255], [255, 255, 255], [255, 255,...","[[37, 70], [34, 56], [35, 61], [35, 37], [37, ..."
